# 📈 3-Day Forecast for Tickers in CSV
This notebook reads tickers from an uploaded CSV and forecasts the next 3 closing prices for each ticker using LSTM.

In [ ]:
# This line executes: !pip install yfinance tensorflow scikit-learn --quiet
!pip install yfinance tensorflow scikit-learn --quiet


In [ ]:
# This line executes: import yfinance as yf
import yfinance as yf
# This line executes: import pandas as pd
import pandas as pd
# This line executes: import numpy as np
import numpy as np
# This line executes: from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
# This line executes: from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
# This line executes: from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import LSTM, Dense, Dropout
# This line executes: from google.colab import files
from google.colab import files


In [ ]:
# Upload the CSV file that contains the 'Ticker' column
# This line executes: uploaded = files.upload()
uploaded = files.upload()


Saving Companies_Sentiment_Analysis_7_Days_Track.csv to Companies_Sentiment_Analysis_7_Days_Track.csv


In [ ]:
# Load the CSV
# This line executes: import io
import io

# This line executes: csv_filename = list(uploaded.keys())[0]
csv_filename = list(uploaded.keys())[0]
# This line executes: df = pd.read_csv(io.BytesIO(uploaded[csv_filename]))
df = pd.read_csv(io.BytesIO(uploaded[csv_filename]))

# Extract unique tickers
# This line executes: tickers = df['Ticker'].unique().tolist()
tickers = df['Ticker'].unique().tolist()
# This line executes: print(f"Found {len(tickers)} unique tickers.")
print(f"Found {len(tickers)} unique tickers.")
# This line executes: print(tickers[:10])  # Preview first 10
print(tickers[:10])  # Preview first 10


Found 179 unique tickers.
['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'META', 'NFLX', 'AMD', 'INTC']


In [ ]:
# This line executes: def fetch_stock_data(ticker, period="1y"):
def fetch_stock_data(ticker, period="1y"):
# This line executes: try:
    try:
# This line executes: df = yf.download(ticker, period=period)
        df = yf.download(ticker, period=period)
# This line executes: return df[["Close"]]
        return df[["Close"]]
# This line executes: except:
    except:
# This line executes: return pd.DataFrame()
        return pd.DataFrame()

# This line executes: def preprocess_data(df, seq_length=60):
def preprocess_data(df, seq_length=60):
# This line executes: scaler = MinMaxScaler()
    scaler = MinMaxScaler()
# This line executes: scaled_data = scaler.fit_transform(df)
    scaled_data = scaler.fit_transform(df)

# This line executes: X, y = [], []
    X, y = [], []
# This line executes: for i in range(seq_length, len(scaled_data)):
    for i in range(seq_length, len(scaled_data)):
# This line executes: X.append(scaled_data[i-seq_length:i, 0])
        X.append(scaled_data[i-seq_length:i, 0])
# This line executes: y.append(scaled_data[i, 0])
        y.append(scaled_data[i, 0])

# This line executes: X = np.array(X)
    X = np.array(X)
# This line executes: y = np.array(y)
    y = np.array(y)
# This line executes: X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# This line executes: return X, y, scaler
    return X, y, scaler

# This line executes: def build_model():
def build_model():
# This line executes: model = Sequential()
    model = Sequential()
# This line executes: model.add(LSTM(50, return_sequences=True, input_shape=(60, 1)))
    model.add(LSTM(50, return_sequences=True, input_shape=(60, 1)))
# This line executes: model.add(Dropout(0.2))
    model.add(Dropout(0.2))
# This line executes: model.add(LSTM(50))
    model.add(LSTM(50))
# This line executes: model.add(Dropout(0.2))
    model.add(Dropout(0.2))
# This line executes: model.add(Dense(1))
    model.add(Dense(1))
# This line executes: model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(optimizer='adam', loss='mean_squared_error')
# This line executes: return model
    return model

# This line executes: def forecast_next_3_days(ticker):
def forecast_next_3_days(ticker):
# This line executes: df = fetch_stock_data(ticker)
    df = fetch_stock_data(ticker)
# This line executes: if df.empty or len(df) < 100:
    if df.empty or len(df) < 100:
# This line executes: return ["N/A", "N/A", "N/A"]
        return ["N/A", "N/A", "N/A"]

# This line executes: X, y, scaler = preprocess_data(df)
    X, y, scaler = preprocess_data(df)
# This line executes: model = build_model()
    model = build_model()
# This line executes: model.fit(X, y, epochs=5, batch_size=32, verbose=0)
    model.fit(X, y, epochs=5, batch_size=32, verbose=0)

# This line executes: last_60 = df[-60:].values
    last_60 = df[-60:].values
# This line executes: last_60_scaled = scaler.transform(last_60)
    last_60_scaled = scaler.transform(last_60)
# This line executes: forecast_input = last_60_scaled.reshape(1, 60, 1)
    forecast_input = last_60_scaled.reshape(1, 60, 1)

# This line executes: predictions = []
    predictions = []
# This line executes: for _ in range(3):
    for _ in range(3):
# This line executes: next_price = model.predict(forecast_input)[0][0]
        next_price = model.predict(forecast_input)[0][0]
# This line executes: predictions.append(next_price)
        predictions.append(next_price)
# This line executes: forecast_input = np.append(forecast_input[:, 1:, :], [[[next_price]]], axis=1)
        forecast_input = np.append(forecast_input[:, 1:, :], [[[next_price]]], axis=1)

# This line executes: predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
# This line executes: return [round(p, 2) for p in predictions]
    return [round(p, 2) for p in predictions]


In [ ]:
# This line executes: results = []
results = []

# This line executes: for ticker in tickers:
for ticker in tickers:
# This line executes: print(f"Forecasting: {ticker}")
    print(f"Forecasting: {ticker}")
# This line executes: forecast = forecast_next_3_days(ticker)
    forecast = forecast_next_3_days(ticker)
# This line executes: results.append({
    results.append({
# This line executes: "Ticker": ticker,
        "Ticker": ticker,
# This line executes: "Forecast Day 1": forecast[0],
        "Forecast Day 1": forecast[0],
# This line executes: "Forecast Day 2": forecast[1],
        "Forecast Day 2": forecast[1],
# This line executes: "Forecast Day 3": forecast[2]
        "Forecast Day 3": forecast[2]
# This line executes: })
    })

# This line executes: forecast_df = pd.DataFrame(results)
forecast_df = pd.DataFrame(results)
# This line executes: forecast_df.to_csv("Forecast_3_Days_Per_Ticker.csv", index=False)
forecast_df.to_csv("Forecast_3_Days_Per_Ticker.csv", index=False)
# This line executes: print("Saved to Forecast_3_Days_Per_Ticker.csv")
print("Saved to Forecast_3_Days_Per_Ticker.csv")

# This line executes: from IPython.display import display
from IPython.display import display
# This line executes: display(forecast_df.head())
display(forecast_df.head())


Forecasting: AAPL


/tmp/ipython-input-775772355.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Forecasting: MSFT


/tmp/ipython-input-775772355.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Forecasting: GOOG


...

/tmp/ipython-input-775772355.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Forecasting: IR


/tmp/ipython-input-775772355.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=period)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Saved to Forecast_3_Days_Per_Ticker.csv


,Ticker,Forecast Day 1,Forecast Day 2,Forecast Day 3
0,AAPL,214.990005,214.919998,215.000000
1,MSFT,522.849976,524.070007,525.130005
2,GOOG,190.190002,190.449997,190.589996
3,AMZN,223.080002,222.610001,222.220001
4,TSLA,316.570007,316.239990,316.010010


In [ ]:
# This line executes: from google.colab import files
from google.colab import files
# This line executes: files.download("Forecast_3_Days_Per_Ticker.csv")
files.download("Forecast_3_Days_Per_Ticker.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>